In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True, limit=500000)

--2020-09-14 13:12:16--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.18.78
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.18.78|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
word2vec.word_vec("test").shape

(300,)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, Embedding 
from tensorflow.keras.models import Model

In [ ]:
! ls drive

'My Drive'


In [ ]:
alldata = pd.read_csv('/content/drive/My Drive/news_dataset.csv')

In [ ]:
alldata.head()

,Unnamed: 0,title,content,publication,label
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake


In [ ]:
alldata = alldata.drop(['Unnamed: 0', 'publication'], axis=1)

In [ ]:
alldata['label'].replace({'fake':1, 'real':0}, inplace=True)
alldata.isnull().values.any()
alldata.dropna(inplace=True)

In [ ]:
alldata.head()

,title,content,label
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,1
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,1
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,1
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,1
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,1


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
word_tokenize(alldata['content'][0])

['Print',
 'They',
 'should',
 'pay',
 'all',
 'the',
 'back',
 'all',
 'the',
 'money',
 'plus',
 'interest',
 '.',
 'The',
 'entire',
 'family',
 'and',
 'everyone',
 'who',
 'came',
 'in',
 'with',
 'them',
 'need',
 'to',
 'be',
 'deported',
 'asap',
 '.',
 'Why',
 'did',
 'it',
 'take',
 'two',
 'years',
 'to',
 'bust',
 'them',
 '?',
 'Here',
 'we',
 'go',
 'again',
 '…another',
 'group',
 'stealing',
 'from',
 'the',
 'government',
 'and',
 'taxpayers',
 '!',
 'A',
 'group',
 'of',
 'Somalis',
 'stole',
 'over',
 'four',
 'million',
 'in',
 'government',
 'benefits',
 'over',
 'just',
 '10',
 'months',
 '!',
 'We',
 '’',
 've',
 'reported',
 'on',
 'numerous',
 'cases',
 'like',
 'this',
 'one',
 'where',
 'the',
 'Muslim',
 'refugees/immigrants',
 'commit',
 'fraud',
 'by',
 'scamming',
 'our',
 'system…It',
 '’',
 's',
 'way',
 'out',
 'of',
 'control',
 '!',
 'More',
 'Related']

In [ ]:
newcontent = []
allcontent = alldata['content'].values
for cont in allcontent:
  aall = word_tokenize(cont)
  ball = [a.lower() for a in aall if a.isalpha()]
  newcontent.append(' '.join(ball))

In [ ]:
alldata['clean content'] = newcontent

In [ ]:
alldata.head()

,title,content,label,clean content
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,1,print they should pay all the back all the mon...
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,1,why did attorney general loretta lynch plead t...
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,1,red state fox news sunday reported this mornin...
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,1,email kayla mueller was a prisoner and torture...
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,1,email healthcare reform to make america great ...


In [ ]:
clean_data = alldata[['label', 'clean content']]

In [ ]:
clean_data.to_csv('clean_content.csv')

In [ ]:
df_train, df_test, Ytrain, Ytest = train_test_split(np.array(newcontent), alldata['label'], test_size=0.33)

In [ ]:
cleandata = pd.read_csv('clean_content.csv')

In [ ]:
cleandata = cleandata[cleandata['clean content'].apply(type)==str]

In [ ]:
df_train, df_test, Ytrain, Ytest = train_test_split(cleandata['clean content'], cleandata['label'], test_size=0.33)

In [ ]:
MAX_VOCAB_SIZE = 200000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train.values)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

In [ ]:
V = len(tokenizer.word_index) + 1 

In [ ]:
V

155674

In [ ]:
data_train = pad_sequences(sequences_train)

In [ ]:
T = data_train.shape[1]
T

29286

In [ ]:
data_test = pad_sequences(sequences_test, maxlen=T)

In [ ]:
embedding_matrix = np.zeros((V , 300))
for word,ind in tokenizer.word_index.items():
  try:
    embedding_matrix[ind,:] = word2vec.word_vec(word).reshape(300)
  except:
    pass

In [ ]:
embedding_matrix.shape

(155674, 300)

In [ ]:
M = 10


i = Input(shape=(T,))
x = Embedding(V, 300, weights=[embedding_matrix], trainable=False)(i)

# Now we have N x T x D

x = LSTM(M, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

In [ ]:
model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
r = model.fit(
  data_train,
  Ytrain,
  epochs=10,
  validation_data=(data_test, Ytest),
  batch_size=50
)

Epoch 1/10


ResourceExhaustedError: ignored

In [ ]:
cleandata.to_csv('new_clean_content.csv')

In [ ]:
np.save('embmat.npy', embedding_matrix)

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)